In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [54]:
#path="C:\\Dropbox\\Google\\MSc BA\\Courses\\AML\\Kaggle\\Telstra\\Data\\"
path="C:\\Users\\stan\\Google Drive\\MSc BA\\Courses\\AML\\Kaggle\\Telstra\\Data\\"

trainFile="train.csv"
testFile="test.csv"
resourceFile="resource_type.csv"
eventTypeFile="event_type.csv"
logFeatureFile="log_feature.csv"
severityTypeFile="severity_type.csv"

test=pd.read_csv(filepath_or_buffer=path+testFile,delimiter=",",header=0)
train=pd.read_csv(filepath_or_buffer=path+trainFile,delimiter=",",header=0)

resource=pd.read_csv(filepath_or_buffer=path+resourceFile,delimiter=",",header=0)
event=pd.read_csv(filepath_or_buffer=path+eventTypeFile,delimiter=",",header=0)
feature=pd.read_csv(filepath_or_buffer=path+logFeatureFile,delimiter=",",header=0)
severity=pd.read_csv(filepath_or_buffer=path+severityTypeFile,delimiter=",",header=0)
join=pd.DataFrame({'id':[]})
features={'event':event,'feature':feature,'severity':severity} #add 'resource':resource back
notCat=['id','volume']

In [ ]:
#convert table of ids and classes to binary row feature vectors for each unique id
#takes a long time, thus once computed save results to csv file
for key, feature in features.items():
    for column in feature.columns:
        if column not in notCat:
            tmp=pd.get_dummies(feature[column],sparse=True)
            tmp['id']=feature['id']
            tmp=tmp.groupby('id').sum()
            feature=tmp
            features[key]=feature
            filename='new '+str(key)+'.csv'
            feature.to_csv(filename)      

In [3]:
#creates join dataset table that will store features for train and test datasets
train['sample']='train'
test['sample']='test'
test['fault_severity']=np.nan
join=pd.concat([train,test],ignore_index=True)

array([    1,     1,     2, ..., 18550, 18551, 18552], dtype=int64)

In [4]:
#joins features from different datasets into one dataframe, features only for evaluated ids are extracted
#(i.e. some features may be not related to any id)
for key,dataset in features.items():
        join=pd.merge(join,dataset,on='id',how='left')

join=join.set_index(['sample',join.index])

In [5]:
#retired
import collections
cat=collections.namedtuple('CategoricalConverter',['map','numSeries'])
    
def catSeries2numSeries(catSeries):
    cats=np.sort(catSeries.unique())
    cat_mapping=dict(zip(cats,range(0,len(cats)+1)))
    numSeries=catSeries.map(cat_mapping).astype(int)
    obs=cat(cat_mapping,numSeries)
    return obs

catLabels=join.dtypes[join.dtypes.map(lambda x: x == 'object')]
catLabels=catLabels.index
cat_map={}
counter=0
for label in catLabels:
    convert=catSeries2numSeries(join[label])
    cat_map[label]=convert.map #attribute from CategoricalConverter
    join[label]=convert.numSeries
    #setting categorical mapping; this is bogus, since vars are categorical

In [8]:
from sklearn.ensemble import RandomForestClassifier

features=['location','log_feature','severity_type','event_type', 'resource_type','volume']

train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

# Fit the model to training data
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(train_features, train_target)

score = clf.score(train_features, train_target)
"Mean accuracy of Random Forest: {0}".format(score)

test_response=clf.predict_proba(test_features)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

# Split 80-20 train vs test data
train_x, test_x, train_y, test_y = train_test_split(train_features, train_target, test_size=0.31, random_state=0)

clf = clf.fit(train_x, train_y)
predict_y = clf.predict_proba(test_x)

print ("LogLoss = %.2f" % (log_loss(test_y, predict_y)))

In [26]:
#save predictions to required format
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=test['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
meanOut.to_csv('predictions.csv',index=False)